In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from scipy import stats
import math
import pickle

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
woba_prediction, zr_prediction, *pitching_models = pickle.load(open('projection_models.p', 'rb'))

In [3]:
path = "C:/Users/gonzo/Documents/Out of the Park Developments/OOTP Baseball 23/saved_games/Shredders.lg/import_export/csv/"

# Current year, and my team
current_year = pd.read_csv(path + 'players_career_batting_stats.csv')['year'].max()
teams = pd.read_csv(path + 'teams.csv')
teams['team'] = teams['name'] + ' ' + teams['nickname']
my_team_id = teams[teams['human_team'] == 1].reset_index().at[0, 'team_id']

# One-hot encode Players data
plyrs = pd.read_csv(path + 'players.csv', usecols = ['player_id', 'team_id', 'age', 'throws', 'bats', 'height', 'personality_greed', 'personality_loyalty', 'personality_play_for_winner', 'personality_work_ethic', 'personality_intelligence', 'personality_leader'])
plyrs['throws_left'] = np.where(plyrs['throws'] == 2, 1, 0)
plyrs['throws_right'] = np.where(plyrs['throws'] == 1, 1, 0)
plyrs['bats_left'] = np.where(plyrs['bats'] == 2, 1, 0)
plyrs['bats_right'] = np.where(plyrs['bats'] == 1, 1, 0)
plyrs['bats_switch'] = np.where(plyrs['bats'] == 3, 1, 0)
plyrs.drop(['throws', 'bats'], axis = 'columns', inplace = True)

bench_coach_flds = ['teach_hitting', 'teach_pitching', 'teach_c', 'teach_if', 'teach_of', 'teach_running', 'handle_veterans', 'handle_rookies', 'handle_players', 'favor_speed_to_power', 'favor_avg_to_obp', 'favor_defense_to_offense', 'favor_pitching_to_hitting', 'favor_veterans_to_prospects', 'manager_value']
hitting_coach_flds = ['teach_hitting', 'handle_veterans', 'handle_rookies', 'handle_players', 'favor_speed_to_power', 'favor_avg_to_obp', 'favor_defense_to_offense', 'favor_pitching_to_hitting', 'favor_veterans_to_prospects', 'hitting_coach_value',  'hitting_focus']
pitching_coach_flds = ['teach_pitching', 'handle_veterans', 'handle_rookies', 'handle_players', 'favor_speed_to_power', 'favor_avg_to_obp', 'favor_defense_to_offense', 'favor_pitching_to_hitting', 'favor_veterans_to_prospects', 'pitching_coach_value', 'pitching_focus']
added_columns = ['hitting_focus_contact', 'hitting_focus_power', 'hitting_focus_patience', 'hitting_focus_neutral', 'pitching_focus_power','pitching_focus_finesse','pitching_focus_groundball','pitching_focus_neutral', 'personality_personable','personality_easygoing','personality_normal','personality_tempermental','personality_controlling']

# One-hot encode my team's coach data
my_coaches = pd.read_csv(path + 'coaches.csv').query('team_id == ' + str(my_team_id))

my_coaches['hitting_focus_contact'] = np.where(my_coaches['hitting_focus'] == 0, 1, 0)
my_coaches['hitting_focus_power'] = np.where(my_coaches['hitting_focus'] == 1, 1, 0)
my_coaches['hitting_focus_patience'] = np.where(my_coaches['hitting_focus'] == 2, 1, 0)
my_coaches['hitting_focus_neutral'] = np.where(my_coaches['hitting_focus'] == 3, 1, 0)

my_coaches['pitching_focus_power'] = np.where(my_coaches['pitching_focus'] == 0, 1, 0)
my_coaches['pitching_focus_finesse'] = np.where(my_coaches['pitching_focus'] == 1, 1, 0)
my_coaches['pitching_focus_groundball'] = np.where(my_coaches['pitching_focus'] == 2, 1, 0)
my_coaches['pitching_focus_neutral'] = np.where(my_coaches['pitching_focus'] == 3, 1, 0)

my_coaches['personality_personable'] = np.where(my_coaches['personality'] == 0, 1, 0)
my_coaches['personality_easygoing'] = np.where(my_coaches['personality'] == 1, 1, 0)
my_coaches['personality_normal'] = np.where(my_coaches['personality'] == 2, 1, 0)
my_coaches['personality_tempermental'] = np.where(my_coaches['personality'] == 3, 1, 0)
my_coaches['personality_controlling'] = np.where(my_coaches['personality'] == 4, 1, 0)
            
bc = my_coaches.query('occupation == 3')[['team_id'] + bench_coach_flds + added_columns]
hc = my_coaches.query('occupation == 5')[['team_id'] + hitting_coach_flds + added_columns]
pc = my_coaches.query('occupation == 4')[['team_id'] + pitching_coach_flds + added_columns]

# My team's coach dataframe
my_coach_data = bc.merge(hc, how = 'inner', on = 'team_id', suffixes = ['_bench_coach', '_hitting_coach']).merge(pc, how = 'inner',  on = 'team_id', suffixes = ['_bench_coach', '_pitching_coach']).drop('team_id', axis = 'columns')

# number of pitches
pitches = ['Fastball', 'Slider', 'Curveball', 'Screwball', 'Forkball', 'Changeup', 'Sinker', 'Splitter', 'Knuckleball', 'Cutter', 'Circlechange', 'Knucklecurve']
get_num_pitches = np.vectorize(lambda pitches : np.count_nonzero(pitches, axis = 1), excluded = [0])

# Batting Fields
main_batting_fields = ['Contact', 'Gap', 'Power', 'Eye', 'Avoid Ks', 'BABIP']
vsr_batting_fields = ['vsR ' + fld for fld in main_batting_fields]
vsl_batting_fields = ['vsL ' + fld for fld in main_batting_fields]
ovr_batting_fields = ['Ovr ' + fld for fld in main_batting_fields]
pot_batting_fields = ['Pot ' + fld for fld in main_batting_fields]
other_batting_fields = ['age', 'GB Hitter Type', 'FB Hitter Type', 'bats_left', 'bats_right', 'bats_switch', 'personality_greed', 'personality_loyalty', 'personality_play_for_winner', 'personality_work_ethic', 'personality_intelligence', 'personality_leader']
coach_fields_hitting = ['teach_hitting_bench_coach', 'teach_running', 'handle_veterans_bench_coach', 'handle_rookies_bench_coach', 'handle_players_bench_coach', 'favor_speed_to_power_bench_coach', 'favor_avg_to_obp_bench_coach', 'favor_defense_to_offense_bench_coach', 'favor_pitching_to_hitting_bench_coach', 'favor_veterans_to_prospects_bench_coach', 'manager_value', 'hitting_focus_contact_bench_coach', 'hitting_focus_power_bench_coach', 'hitting_focus_patience_bench_coach', 'hitting_focus_neutral_bench_coach', 'pitching_focus_power_bench_coach', 'pitching_focus_finesse_bench_coach', 'pitching_focus_groundball_bench_coach', 'pitching_focus_neutral_bench_coach', 'personality_personable_bench_coach', 'personality_easygoing_bench_coach', 'personality_normal_bench_coach', 'personality_tempermental_bench_coach','personality_controlling_bench_coach', 'teach_hitting_hitting_coach','handle_veterans_hitting_coach', 'handle_rookies_hitting_coach', 'handle_players_hitting_coach', 'favor_speed_to_power_hitting_coach', 'favor_avg_to_obp_hitting_coach', 'favor_defense_to_offense_hitting_coach', 'favor_pitching_to_hitting_hitting_coach', 'favor_veterans_to_prospects_hitting_coach', 'hitting_coach_value', 'hitting_focus', 'hitting_focus_contact_hitting_coach', 'hitting_focus_power_hitting_coach', 'hitting_focus_patience_hitting_coach', 'hitting_focus_neutral_hitting_coach', 'pitching_focus_power_hitting_coach', 'pitching_focus_finesse_hitting_coach', 'pitching_focus_groundball_hitting_coach', 'pitching_focus_neutral_hitting_coach', 'personality_personable_hitting_coach', 'personality_easygoing_hitting_coach', 'personality_normal_hitting_coach', 'personality_tempermental_hitting_coach', 'personality_controlling_hitting_coach']

running_fields = ['Speed', 'Steal', 'Baserunning']

# Pitching Fields
main_pitching_fields = ['Stuff', 'Movement', 'Control']
vsr_pitching_fields = ['vsR ' + fld for fld in main_pitching_fields]
vsl_pitching_fields = ['vsL ' + fld for fld in main_pitching_fields]
pot_pitching_fields = ['Pot ' + fld for fld in main_pitching_fields]
# osa_pitches = [pitch + '_osa' for pitch in pitches]
pot_pitches = ['Pot ' + pitch for pitch in pitches]# + ['Pot ' + pitch for pitch in pitches]
other_pitching_fields = ['num_pitches', 'Velocity', 'Arm Slot', 'Stamina', 'Ground/Fly', 'age', 'throws_left', 'throws_right', 'personality_greed', 'personality_loyalty', 'personality_play_for_winner', 'personality_work_ethic', 'personality_intelligence', 'personality_leader']
catcher_abilities = ['wa_CA']
coaches_fields_pitching = ['teach_pitching_bench_coach', 'handle_veterans_bench_coach', 'handle_rookies_bench_coach', 'handle_players_bench_coach', 'favor_speed_to_power_bench_coach', 'favor_avg_to_obp_bench_coach', 'favor_defense_to_offense_bench_coach', 'favor_pitching_to_hitting_bench_coach', 'favor_veterans_to_prospects_bench_coach', 'manager_value', 'hitting_focus_contact_bench_coach', 'hitting_focus_power_bench_coach', 'hitting_focus_patience_bench_coach', 'hitting_focus_neutral_bench_coach', 'pitching_focus_power_bench_coach', 'pitching_focus_finesse_bench_coach', 'pitching_focus_groundball_bench_coach', 'pitching_focus_neutral_bench_coach', 'personality_personable_bench_coach', 'personality_easygoing_bench_coach', 'personality_normal_bench_coach', 'personality_tempermental_bench_coach', 'personality_controlling_bench_coach', 'teach_pitching_pitching_coach', 'handle_veterans', 'handle_rookies', 'handle_players', 'favor_speed_to_power', 'favor_avg_to_obp', 'favor_defense_to_offense', 'favor_pitching_to_hitting', 'favor_veterans_to_prospects', 'pitching_coach_value', 'pitching_focus', 'hitting_focus_contact', 'hitting_focus_power', 'hitting_focus_patience', 'hitting_focus_neutral', 'pitching_focus_power', 'pitching_focus_finesse', 'pitching_focus_groundball', 'pitching_focus_neutral', 'personality_personable', 'personality_easygoing', 'personality_normal', 'personality_tempermental', 'personality_controlling']
pitching_stats = ['fip', 'k_rate', 'bb_rate', 'hr_rate']

# Defensive model fields
positions = ['pitcher', 'catcher', 'first_base', 'second_base', 'third_base', 'shortstop', 'left_field', 'center_field', 'right_field']
main_fields = ['IF Range', 'IF Arm', 'Turn DP', 'OF Range', 'OF Arm', 'Catcher Arm', 'Catcher Ability', 'IF Error', 'OF Error', 'Fld - P', 'Fld - C', 'Fld - 1B', 'Fld - 2B', 'Fld - 3B', 'Fld - SS', 'Fld - LF', 'Fld - CF', 'Fld - RF']
# osa_main_fields = [fld + '_osa' for fld in main_fields]
other_fields = ['height', 'age', 'throws_left', 'throws_right', 'personality_greed', 'personality_loyalty', 'personality_play_for_winner', 'personality_work_ethic', 'personality_intelligence', 'personality_leader']
coach_fields_fielding = ['teach_c', 'teach_if', 'teach_of', 'handle_veterans_bench_coach', 'handle_rookies_bench_coach', 'handle_players_bench_coach', 'favor_speed_to_power_bench_coach', 'favor_avg_to_obp_bench_coach', 'favor_defense_to_offense_bench_coach', 'favor_pitching_to_hitting_bench_coach', 'favor_veterans_to_prospects_bench_coach', 'manager_value', 'hitting_focus_contact_bench_coach', 'hitting_focus_power_bench_coach', 'hitting_focus_patience_bench_coach', 'hitting_focus_neutral_bench_coach', 'pitching_focus_power_bench_coach', 'pitching_focus_finesse_bench_coach', 'pitching_focus_groundball_bench_coach', 'pitching_focus_neutral_bench_coach', 'personality_personable_bench_coach', 'personality_easygoing_bench_coach', 'personality_normal_bench_coach', 'personality_tempermental_bench_coach', 'personality_controlling_bench_coach']

In [4]:
my_teams_catchers = pd.read_csv(path + 'players_scouted_ratings.csv', usecols = ['player_id', 'team_id', 'scouting_coach_id', 'position', 'fielding_ratings_catcher_ability']).query('scouting_coach_id == -1 and position == 2 and team_id == ' + str(my_team_id)).groupby('scouting_coach_id').mean()['fielding_ratings_catcher_ability'].to_list()

# Update Current Ratings with projections

In [5]:
# Get current ratings
current_ratings = pd.read_csv(path + 'players_scouted_ratings.csv', header = 0, names = ['player_id', 'team_id', 'league_id', 'position', 'role', 'scout_id', 'Speed', 'Steal', 'Baserunning', 'Ovr Contact', 'Ovr Gap', 'Ovr Eye', 'Ovr Avoid Ks', 'Ovr Get Hit', 'Ovr Power', 'Ovr BABIP', 'vsR Contact', 'vsR Gap', 'vsR Eye', 'vsR Avoid Ks', 'vsR Get Hit', 'vsR Power', 'vsR BABIP', 'vsL Contact', 'vsL Gap', 'vsL Eye', 'vsL Avoid Ks', 'vsL Get Hit', 'vsL Power', 'vsL BABIP', 'Pot Contact', 'Pot Gap', 'Pot Eye', 'Pot Avoid Ks', 'Pot Get Hit', 'Pot Power', 'Pot BABIP', 'Bunt', 'Bunt for Hit', 'GB Hitter Type', 'FB Hitter Type', 'Ovr Stuff', 'Ovr Control', 'Ovr Movement', 'Ovr Balk', 'Ovr HBP', 'Ovr WP', 'vsR Stuff', 'vsR Control', 'vsR Movement', 'vsR Balk', 'vsR HBP', 'vsR WP', 'vsL Stuff', 'vsL Control', 'vsL Movement', 'vsL Balk', 'vsL HBP', 'vsL WP', 'Pot Stuff', 'Pot Control', 'Pot Movement', 'Pot Balk', 'Pot HBP', 'Pot WP', 'Fastball', 'Slider', 'Curveball', 'Screwball', 'Forkball', 'Changeup', 'Sinker', 'Splitter', 'Knuckleball', 'Cutter', 'Circlechange', 'Knucklecurve', 'Pot Fastball', 'Pot Slider', 'Pot Curveball', 'Pot Screwball', 'Pot Forkball', 'Pot Changeup', 'Pot Sinker', 'Pot Splitter', 'Pot Knuckleball', 'Pot Cutter', 'Pot Circlechange', 'Pot Knucklecurve', 'Velocity', 'Arm Slot', 'Stamina', 'Ground/Fly', 'Hold Runners', 'IF Range', 'IF Arm', 'Turn DP', 'OF Range', 'OF Arm', 'Catcher Arm', 'Catcher Ability', 'IF Error', 'OF Error', 'Fld - P', 'Fld - C', 'Fld - 1B', 'Fld - 2B', 'Fld - 3B', 'Fld - SS', 'Fld - LF', 'Fld - CF', 'Fld - RF', 'Ovr', 'Pot', 'Scouting Accuracy']).query('scout_id != -1').merge(plyrs, how = 'inner', on = 'player_id')
# osa_scout = current_ratings.query('scout_id == -1')
# my_scout = current_ratings.query('scout_id != -1')
# current_ratings = my_scout.merge(osa_scout, how = 'inner', on = 'player_id', suffixes = ['', '_osa']).merge(plyrs, how = 'inner', on = 'player_id')
current_ratings['num_pitches'] = get_num_pitches(current_ratings[pitches].values)

woba_projections = ['woba_projection', 'woba_projection_pot', 'woba_projection_vsr', 'woba_projection_vsl']
fip_projections = ['fip_projection', 'fip_projection_pot', 'fip_projection_vsr', 'fip_projection_vsl', 'k_rate_projection', 'k_rate_projection_vsr', 'k_rate_projection_vsl', 'k_rate_projection_pot', 'bb_rate_projection', 'bb_rate_projection_vsr', 'bb_rate_projection_vsl', 'bb_rate_projection_pot', 'hr_rate_projection', 'hr_rate_projection_vsr', 'hr_rate_projection_vsl', 'hr_rate_projection_pot']
ovr_pit_projections = ['fip_projection', 'k_rate_projection', 'bb_rate_projection', 'hr_rate_projection']
pot_pit_projections = ['fip_projection_pot', 'k_rate_projection_pot', 'bb_rate_projection_pot', 'hr_rate_projection_pot']
zr_projections = ['zr_projection_1', 'zr_projection_2', 'zr_projection_3', 'zr_projection_4', 'zr_projection_5', 'zr_projection_6', 'zr_projection_7', 'zr_projection_8', 'zr_projection_9']

# Offensive Projections
vsr = current_ratings[vsr_batting_fields + other_batting_fields]
vsl = current_ratings[vsl_batting_fields + other_batting_fields]
pot = current_ratings[pot_batting_fields + other_batting_fields]
num_rows = vsr.shape[0]
my_coaches = pd.DataFrame(list(my_coach_data[coach_fields_hitting].values) * num_rows, columns = coach_fields_hitting)

current_ratings['woba_projection_vsr'] = woba_prediction.predict(pd.concat([vsr, my_coaches], axis = 1).values).round(3)
current_ratings['woba_projection_vsl'] = woba_prediction.predict(pd.concat([vsl, my_coaches], axis = 1).values).round(3)
current_ratings['woba_projection'] = (current_ratings['woba_projection_vsr'] * 0.7 + current_ratings['woba_projection_vsl'] * 0.3).round(3)
current_ratings['woba_projection_pot'] = woba_prediction.predict(pd.concat([pot, my_coaches], axis = 1).values).round(3)

# num_rows = ovr_runs.shape[0]
# my_coaches = pd.DataFrame(list(my_coach_data[coach_fields_hitting].values) * num_rows, columns = coach_fields_hitting)

# Pitching Projections
vsr = current_ratings[vsr_pitching_fields + pitches + other_pitching_fields]
vsl = current_ratings[vsl_pitching_fields + pitches + other_pitching_fields]
pot = current_ratings[pot_pitching_fields + pot_pitches + other_pitching_fields]
num_rows = vsr.shape[0]

my_coaches = pd.DataFrame(list(my_coach_data[coaches_fields_pitching].values) * num_rows, columns = coaches_fields_pitching)
my_catchers = pd.DataFrame(np.array(my_teams_catchers * num_rows).reshape(num_rows, 1), columns = catcher_abilities)
my_coaches_and_catchers = pd.concat([my_catchers, my_coaches], axis = 1)

# Apply Pitching Stat projections
for i, stat in enumerate(pitching_stats):
    current_ratings[stat + '_projection_vsr'] = pitching_models[i].predict(pd.concat([vsr, my_coaches_and_catchers], axis = 1).values).round(3)
    current_ratings[stat + '_projection_vsl'] = pitching_models[i].predict(pd.concat([vsl, my_coaches_and_catchers], axis = 1).values).round(3)
    current_ratings[stat + '_projection'] = (current_ratings[stat + '_projection_vsr'] * 0.5 + current_ratings[stat + '_projection_vsl'] * 0.5).round(3)
    current_ratings[stat + '_projection_pot'] = pitching_models[i].predict(pd.concat([pot, my_coaches_and_catchers], axis = 1).values).round(3)
    
# Defensive Projections
# Get the standard columns for fielding that are already in current ratings dataframe
current_ratings_fld = current_ratings[main_fields + other_fields]

# Get number of rows in current ratings dataframe (used to extend coach and position data)
num_rows = current_ratings_fld.shape[0]

# Add my team's coach data to current ratings (project every player as if my coaches were their coaches)
extended_coach_df = pd.DataFrame(list(my_coach_data[coach_fields_fielding].values) * num_rows, columns = coach_fields_fielding)

# calculate zr for each position
for pos in range(1, 10):
    
        # create one-hot-encoding for each position
        pos_array = [0 for i in range(9)]
        pos_array[pos - 1] = 1
        
        # Create the pos_array for every row in the ratings dataframe
        pos_df = pd.DataFrame(list([pos_array]) * num_rows)
        
        # Combine the pos_array with the other data to create the input variables
        prediction_data = pd.concat([pos_df, current_ratings_fld, extended_coach_df], axis = 1).values
        
        # create a new column for each position, display projected zr for that player at that position
        current_ratings['zr_projection_' + str(pos)] = zr_prediction.predict(prediction_data).round(1)

ratings_to_keep = ['Catcher Ability', 'Stamina']
convert_rating_to_20_80_scale = np.vectorize(lambda num : min(80, math.ceil((num - 15) / 16) * 5 + 20))

for att in ratings_to_keep:
    current_ratings[att] = convert_rating_to_20_80_scale(current_ratings[att])

current_ratings = current_ratings[['player_id', 'num_pitches', 'IF Range', 'OF Range', 'IF Arm', 'Turn DP', 'OF Arm'] + ratings_to_keep + woba_projections + fip_projections + zr_projections]

players = pd.read_csv(path + 'players.csv')
players['name'] = players.first_name + ' ' + players.last_name

# Get player service times
service_time = pd.read_csv(path + 'players_roster_status.csv', usecols = ['player_id', 'mlb_service_days', 'trade_status', 'is_on_secondary'])
service_time['service_time'] = service_time['mlb_service_days'] / 172
service_time['Service Time'] = np.where(service_time['service_time'] < 3, 'Pre-Arb', np.where(service_time['service_time'] < 6, 'Arbitration', 'Post Arb'))
service_time.drop('mlb_service_days', axis = 'columns', inplace = True)

# Get player salaries
salaries = pd.read_csv(path + 'players_contract.csv', usecols = ['player_id', 'salary0', 'salary1', 'salary2', 'salary3', 'salary4', 'salary5', 'salary6', 'salary7', 'salary8', 'salary9', 'current_year'])
salaries['current_salary'] = salaries.apply(lambda row: (row['salary' + str(row['current_year'])] / 1000000).round(3), axis = 'columns')
salaries.drop(['salary0', 'salary1', 'salary2', 'salary3', 'salary4', 'salary5', 'salary6', 'salary7', 'salary8', 'salary9', 'current_year'], axis = 'columns', inplace = True)

# Merge for service times, salaries, and waiver status
players = players.merge(service_time, how = 'left', on = 'player_id').merge(salaries, how = 'left', on = 'player_id').merge(pd.read_csv(path + 'players_roster_status.csv', usecols = ['player_id', 'is_on_waivers']).query('is_on_waivers == 1'), how = 'left', on = 'player_id')
players['injured'] = np.where(players['injury_is_injured'] == 1, 'Yes', '')

level_names = {200:'MLB', 201:'AAA', 202:'AA', 203:'A', 204:'Rookie', -200:'Intl Comp'}
get_levels = np.vectorize(level_names.get)
players['level'] = get_levels(players['league_id'])

current_players = players.merge(current_ratings, how = 'inner', on = 'player_id')

teams['parent_team_id'] = np.where(teams['parent_team_id'] == 0, teams['team_id'], teams['parent_team_id'])
current_players = current_players.merge(teams[['team_id', 'parent_team_id', 'team']], how = 'left', on = 'team_id')
current_players = current_players.merge(teams[['team_id', 'team']], left_on = 'parent_team_id', right_on = 'team_id', how = 'left', suffixes = ['', '_org'])
current_players.rename(columns={'team_org': 'org'}, inplace = True)

current_players.rename(columns={'personality_work_ethic':'work_ethic', 'personality_intelligence':'intelligence'}, inplace = True)
current_players['work_ethic'] = convert_rating_to_20_80_scale(current_players['work_ethic'])
current_players['intelligence'] = convert_rating_to_20_80_scale(current_players['intelligence'])
    
def work_ethic_groups(work_ethic):
    if work_ethic <= 35:
        return 'Low'
    elif work_ethic <= 60:
        return 'Normal'
    else:
        return 'High'

def age_groups(age):
    if age <= 21:
        return '16 - 21'
    elif age <= 25:
        return '22 - 25'
    elif age <= 29:
        return '26 - 29'
    elif age <= 33:
        return '30 - 33'
    else:
        return '34+'
    
def level_names(league_id):
    if league_id == -1:
        return 'Intl Comp'
    elif league_id == 0:
        return 'Free Agent'
    elif league_id == 200:
        return 'MLB'
    elif league_id == 201:
        return 'AAA'
    elif league_id == 202:
        return 'AA'
    elif league_id == 203:
        return 'A'
#     elif league_id == 103:
#         return 'A-'
    else:
        return 'Rookie'
    
group_ages = np.vectorize(age_groups)
work_ethic_grouping = np.vectorize(work_ethic_groups)
current_players['Age Group'] = group_ages(current_players['age'])
current_players['Work Ethic'] = work_ethic_grouping(current_players['work_ethic'])
current_players['WE_int'] = np.where(current_players['Work Ethic'] == 'Low', 0, np.where(current_players['Work Ethic'] == 'Normal', 1, 2))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

# Team Performance Dashboard

In [ ]:
def team_dashboard(team_search = 'Denver'):
    search_team_id = teams.query('name == "' + str(team_search) + '" or nickname == "' + str(team_search) + '" or team == "' + str(team_search) + '"').reset_index().at[0, 'team_id']
    search_team_name = teams.query('name == "' + str(team_search) + '" or nickname == "' + str(team_search) + '" or team == "' + str(team_search) + '"').reset_index().at[0, 'team']
    league = teams.query('team_id == ' + str(search_team_id)).iloc[0]['league_id']

    current_year = pd.read_csv(path + 'players_career_batting_stats.csv')['year'].max()
    players = pd.read_csv(path + 'players.csv')
    players['name'] = players.first_name + ' ' + players.last_name

    tm_bat = pd.read_csv(path + 'players_career_batting_stats.csv').query('league_id == ' + str(league) + ' and split_id == 1 and year == ' + str(current_year)).groupby('team_id', as_index = False).sum().merge(teams[['team_id', 'team']], how = 'inner', on = 'team_id')
    tm_bat['s'] = tm_bat['h'] - tm_bat['d'] - tm_bat['t'] - tm_bat['hr']
    tm_bat['woba'] = (tm_bat['bb'] * 0.69 + tm_bat['hp'] * 0.72 + tm_bat['s'] * 0.89 + tm_bat['d'] * 1.27 + tm_bat['t'] * 1.62 + tm_bat['hr'] * 2.1) / (tm_bat['ab'] + tm_bat['bb'] - tm_bat['ibb'] + tm_bat['sf'] + tm_bat['hp'])
    tm_bat.sort_values('woba', ascending = True, inplace = True)
    mean_woba = tm_bat['woba'].mean()

    tm_pit = pd.read_csv(path + 'players_career_pitching_stats.csv').query('league_id == ' + str(league) + ' and split_id == 1 and year == ' + str(current_year)).groupby('team_id', as_index = False).sum().merge(teams[['team_id', 'team']], how = 'inner', on = 'team_id')
    tm_pit['fip'] = (tm_pit['hra'] * 13 + (tm_pit['bb'] + tm_pit['hp']) * 3 - tm_pit['k'] * 2) / tm_pit['ip'] + 3.2
    tm_pit.sort_values('fip', ascending = False, inplace = True)
    mean_fip = tm_pit['fip'].mean()

    tm_fld = pd.read_csv(path + 'players_career_fielding_stats.csv').query('league_id == ' + str(league) + ' and year == ' + str(current_year)).groupby('team_id', as_index = False).sum().merge(teams[['team_id', 'team']], how = 'inner', on = 'team_id').merge(tm_pit[['team_id', 'ha', 'hra', 'bf', 'bb', 'k', 'hp']], how = 'inner', on = 'team_id')
    tm_fld['eff'] = 1 - ((tm_fld['ha'] + tm_fld['roe'] - tm_fld['hra']) / (tm_fld['bf'] - tm_fld['bb'] - tm_fld['k'] - tm_fld['hp'] - tm_fld['hra']))
    tm_fld.sort_values('eff', ascending = True, inplace = True)
    mean_eff = tm_fld['eff'].mean()

    plyr_bat = pd.read_csv(path + 'players_career_batting_stats.csv').query('year == ' + str(current_year) + ' and team_id == ' + str(search_team_id)).drop('position', axis = 'columns').merge(players[['player_id', 'name', 'team_id', 'organization_id', 'position']], how = 'inner', on = 'player_id', suffixes = ['_stats', '_current'])
    plyr_bat['s'] = plyr_bat['h'] - plyr_bat['d'] - plyr_bat['t'] - plyr_bat['hr']
    plyr_bat['woba'] = (plyr_bat['bb'] * 0.69 + plyr_bat['hp'] * 0.72 + plyr_bat['s'] * 0.89 + plyr_bat['d'] * 1.27 + plyr_bat['t'] * 1.62 + plyr_bat['hr'] * 2.1) / (plyr_bat['ab'] + plyr_bat['bb'] - plyr_bat['ibb'] + plyr_bat['sf'] + plyr_bat['hp'])
    plyr_bat_vsr = plyr_bat.query('split_id == 3')
    plyr_bat_vsl = plyr_bat.query('split_id == 2')
    plyr_bat_splits = plyr_bat_vsr.merge(plyr_bat_vsl[['player_id', 'woba']], how = 'outer', on = 'player_id', suffixes = ['_vsr', '_vsl'])[['player_id', 'name', 'position', 'woba_vsr', 'woba_vsl']].fillna(0)
    plyr_bat.sort_values('woba', ascending = True, inplace = True)

    plyr_pit = pd.read_csv(path + 'players_career_pitching_stats.csv').query('team_id == ' + str(search_team_id) + ' and year == ' + str(current_year)).merge(players[['player_id', 'team_id', 'organization_id', 'role', 'name']], how = 'inner', on = 'player_id', suffixes = ['_stats', '_current'])
    plyr_pit['fip'] = (plyr_pit['hra'] * 13 + (plyr_pit['bb'] + plyr_pit['hp']) * 3 - plyr_pit['k'] * 2) / plyr_pit['ip'] + 3.2
    plyr_pit.sort_values('fip', ascending = False, inplace = True)
    plyr_pit_vsr = plyr_pit.query('split_id == 3')
    plyr_pit_vsl = plyr_pit.query('split_id == 2')
    plyr_pit_splits = plyr_pit_vsr.merge(plyr_pit_vsl[['player_id', 'fip']], how = 'outer', on = 'player_id', suffixes = ['_vsr', '_vsl'])[['player_id', 'name', 'role', 'fip_vsr', 'fip_vsl']].fillna(0)

    pos_fld = pd.read_csv(path + 'players_career_fielding_stats.csv').query('position > 1 and team_id == ' + str(search_team_id) + ' and year == ' + str(current_year)).groupby('position', as_index = False).sum()
    pos_fld.sort_values('position', ascending = True, inplace = True)
    plyr_fld = pd.read_csv(path + 'players_career_fielding_stats.csv').query('position > 1 and team_id == ' + str(search_team_id) + ' and year == ' + str(current_year)).groupby('player_id', as_index = False).sum().merge(players[['player_id', 'team_id', 'organization_id', 'name']], how = 'inner', on = 'player_id', suffixes = ['_stats', '_current']).sort_values('zr', ascending = True)

    plt.subplots(figsize=(30, 25))

    # Team Offense
    plt.subplot(3, 3, 1, facecolor = 'whitesmoke')
    plt.barh(tm_bat['team'], tm_bat['woba'], color = ['orange' if tm == search_team_name else 'silver' for tm in tm_bat['team']])
    plt.xlim([0.25, 0.4])
    plt.ylim(plt.ylim())
    plt.plot([mean_woba, mean_woba], plt.ylim(), linestyle = 'dashed', color = 'gray')
    plt.yticks(fontsize = 14)
    plt.title('Team Offense', size = 25)

    # Player Offense
    plt.subplot(3, 3, 2, facecolor = 'whitesmoke')
    plt.barh(plyr_bat.query('split_id == 1')['name'], plyr_bat.query('split_id == 1')['woba'], color = ['black' if (t != search_team_id and o != search_team_id) else 'mediumpurple' if (o == search_team_id and t != search_team_id) else 'lightblue' for t, o in plyr_bat.query('split_id == 1')[['team_id_current', 'organization_id']].values], edgecolor = 'black')
    plt.yticks(fontsize = 14)
    plt.ylim(plt.ylim())
    plt.plot([mean_woba, mean_woba], plt.ylim(), linestyle = 'dashed', color = 'gray')
    plt.title('Player Offense (wOBA)', size = 25)

    # Player Splits
    plt.subplot(3, 3, 3, facecolor = 'whitesmoke')
    plt.scatter(plyr_bat_splits['woba_vsr'], plyr_bat_splits['woba_vsl'], s = 100, c = 'cornflowerblue')
    plt.xlim([min(plt.xlim()[0], plt.ylim()[0]), max(plt.xlim()[1], plt.ylim()[1])])
    plt.ylim(plt.xlim())
    plt.xlim(plt.xlim())
    plt.plot(plt.xlim(), plt.ylim(), linestyle = 'solid', alpha = 0.35, color = 'midnightblue')
    plt.plot([mean_woba, mean_woba], plt.ylim(), linestyle = 'dashed', color = 'gray')
    plt.plot(plt.xlim(), [mean_woba, mean_woba], linestyle = 'dashed', color = 'gray')
    plt.title('Offensive Splits', size = 25)
    plt.xlabel('vs Right', size = 20)
    plt.ylabel('vs Left', size = 20)

    for i, player in enumerate(plyr_bat_splits['name']):
        x_loc = plyr_bat_splits['woba_vsr'].iloc[i]
        y_loc = plyr_bat_splits['woba_vsl'].iloc[i]
        pos = plyr_bat_splits['position'].iloc[i]

        plt.annotate(player + ' - ' + str(pos), (x_loc, y_loc), (x_loc + 0.015, y_loc), size = 16)

    # Team Pitching
    plt.subplot(3, 3, 4, facecolor = 'whitesmoke')
    plt.barh(tm_pit['team'], tm_pit['fip'], color = ['orange' if tm == search_team_name else 'silver' for tm in tm_pit['team']])
    plt.title('Team Pitching', size = 25)
    plt.yticks(fontsize = 14)
    plt.xlim([3.2, 6])
    plt.ylim(plt.ylim())
    plt.plot([mean_fip, mean_fip], plt.ylim(), linestyle = 'dashed', color = 'gray')

    # Player Pitching
    plt.subplot(3, 3, 5, facecolor = 'whitesmoke')
    plt.barh(plyr_pit.query('split_id == 1')['name'], plyr_pit.query('split_id == 1')['fip'], color = ['black' if (t != search_team_id and o != search_team_id) else 'mediumpurple' if (o == search_team_id and t != search_team_id) else 'lightblue' for t, o in plyr_pit.query('split_id == 1')[['team_id_current', 'organization_id']].values], edgecolor = 'black')
    plt.yticks(fontsize = 14)
    plt.ylim(plt.ylim())
    plt.plot([mean_fip, mean_fip], plt.ylim(), linestyle = 'dashed', color = 'gray')
    plt.title('Player Pitching (FIP)', size = 25)

    # Pitching Splits
    plt.subplot(3, 3, 6, facecolor = 'whitesmoke')
    plt.scatter(plyr_pit_splits.query('role == 11')['fip_vsr'], plyr_pit_splits.query('role == 11')['fip_vsl'], s = 100, c = 'cornflowerblue', edgecolor = 'silver', label = 'Starter')
    plt.scatter(plyr_pit_splits.query('role > 11')['fip_vsr'], plyr_pit_splits.query('role > 11')['fip_vsl'], s = 100, c = 'red', edgecolor = 'black', label = 'Reliever')
    plt.xlim([min(plt.xlim()[0], plt.ylim()[0]), max(plt.xlim()[1], plt.ylim()[1])])
    plt.ylim(plt.xlim())
    plt.plot(plt.xlim(), plt.ylim(), linestyle = 'solid', alpha = 0.35, color = 'midnightblue')
    plt.plot([mean_fip, mean_fip], plt.ylim(), linestyle = 'dashed', color = 'gray')
    plt.plot(plt.xlim(), [mean_fip, mean_fip], linestyle = 'dashed', color = 'gray')
    plt.title('Pitching Splits', size = 25)
    plt.xlabel('vs Right', size = 20)
    plt.ylabel('vs Left', size = 20)
    plt.legend(fontsize = 16)

    role_ref = {11:'Starter', 12:'Reliever', 13:'Reliever'}

    for i, player in enumerate(plyr_pit_splits['name']):
        x_loc = plyr_pit_splits['fip_vsr'].iloc[i]
        y_loc = plyr_pit_splits['fip_vsl'].iloc[i]

        plt.annotate(player, (x_loc, y_loc), (x_loc + 0.3, y_loc), size = 14)

    # Team Defensive Efficiency
    plt.subplot(3, 3, 7, facecolor = 'whitesmoke')
    plt.barh(tm_fld['team'], tm_fld['eff'], color = ['orange' if tm == search_team_name else 'silver' for tm in tm_fld['team']])
    plt.title('Team Defense', size = 25)
    plt.yticks(fontsize = 14)
    plt.xlim([0.6, 0.75])
    plt.ylim(plt.ylim())
    plt.plot([mean_eff, mean_eff], plt.ylim(), linestyle = 'dashed', color = 'gray')

    # Player Defense
    plt.subplot(3, 3, 8, facecolor = 'whitesmoke')
    plt.barh(plyr_fld['name'], plyr_fld['zr'], color = ['black' if (t != search_team_id and o != search_team_id) else 'mediumpurple' if (o == search_team_id and t != search_team_id) else 'lightblue' for t, o in plyr_fld[['team_id_current', 'organization_id']].values], edgecolor = 'black')
    plt.yticks(fontsize = 14)
    plt.title('Player Defense (ZR)', size = 25)

    # Defense by Position
    plt.subplot(3, 3, 9, facecolor = 'whitesmoke')
    plt.bar(pos_fld['position'], pos_fld['zr'], color = 'cornflowerblue')
    plt.xticks(plt.xticks()[0], labels = ['', 'Catcher', 'First Base', 'Second Base', 'Third Base', 'Shortstop', 'Left Field', 'Center Field', 'Right Field', ''], fontsize = 18, rotation = 45, fontstretch = 'expanded')
    plt.title('Positional Defense', size = 25)

    plt.tight_layout(pad=1)
    plt.show()

In [ ]:
def minor_league_team_dashboard(team_search = 'Denver', league_id = 201):
    current_year = pd.read_csv(path + 'players_career_batting_stats.csv')['year'].max()
    org_id = teams.query('name == "' + str(team_search) + '" or nickname == "' + str(team_search) + '" or team == "' + str(team_search) + '"').reset_index().at[0, 'team_id']
    search_team_id = teams.query('parent_team_id == ' + str(org_id) + ' and league_id == ' + str(league_id)).iloc[0]['team_id']

    stats = pd.read_csv(path + 'players_career_batting_stats.csv').query('year == ' + str(current_year) + ' and team_id == ' + str(search_team_id) + ' and split_id == 1')
    p = current_players.query('team_id == ' + str(search_team_id))
    bat = stats.merge(p[['player_id', 'name', 'rank_prospect_max']], how = 'inner', on = 'player_id')
    calculate_off_stats(bat)
    bat.sort_values('woba', inplace = True)

    stats = pd.read_csv(path + 'players_career_pitching_stats.csv').query('year == ' + str(current_year) + ' and team_id == ' + str(search_team_id) + ' and split_id == 1')
    pit = stats.merge(p[['player_id', 'name', 'rank_prospect_max']], how = 'inner', on = 'player_id')
    calculate_pit_stats(pit)
    pit.sort_values('fip', ascending = False, inplace = True)

    plt.subplots(figsize=(30, 15))
    plt.suptitle('Minor League - ' + str(league_id), size = 30)
    plt.subplot(2, 2, 1, facecolor = 'whitesmoke')
    plt.title('wOBA', fontsize = 18)
    plt.barh(bat['name'], bat['woba'], color = 'lightblue', edgecolor = 'black')

    plt.subplot(2, 2, 2, facecolor = 'whitesmoke')
    plt.title('Number of PA', fontsize = 18)
    plt.barh(bat['name'], bat['pa'], color = 'lightblue', edgecolor = 'black')

    plt.subplots(figsize=(30, 15))
    plt.subplot(2, 2, 3, facecolor = 'whitesmoke')
    plt.title('FIP', fontsize = 18)
    plt.barh(pit['name'], pit['fip'], color = 'lightblue', edgecolor = 'black')

    plt.subplot(2, 2, 4, facecolor = 'whitesmoke')
    plt.title('Number of BF', fontsize = 18)
    plt.barh(pit['name'], pit['bf'], color = 'lightblue', edgecolor = 'black')

    plt.show()

# Create Positional Rankings (% rankings)

In [ ]:
pos_value_weights = {
    2: (.25, .75),
    3: (.9, .1),
    4: (.45, .55),
    5: (.57, .43),
    6: (.3, .7),
    7: (.75, .25),
    8: (.32, .68),
    9: (.63, .37)
}

current_players['fip_projection'] = np.where(current_players['num_pitches'] == 1, np.nan, current_players['fip_projection'])
current_players['fip_projection_pot'] = np.where(current_players['num_pitches'] == 1, np.nan, current_players['fip_projection_pot'])

current_players['rank_1'] = 1 - (current_players['fip_projection'] - current_players['fip_projection'].min()) / (current_players['fip_projection'].max() - current_players['fip_projection'].min())
current_players['rank_prospect_1'] = 1 - (current_players['fip_projection_pot'] - current_players['fip_projection_pot'].min()) / (current_players['fip_projection_pot'].max() - current_players['fip_projection_pot'].min())

for pos in range(2, 10):
    fld = 'zr_projection_' + str(pos)
    if pos == 2:
        fld = 'Catcher Ability'
    
    hitting = current_players['woba_projection']
    hitting_pot = current_players['woba_projection_pot']
    fielding = current_players[fld]
    
    current_players['wa_projection_' + str(pos)] = (fielding - fielding.min()) / (fielding.max() - fielding.min()) * pos_value_weights[pos][1] + (hitting - hitting.min()) / (hitting.max() - hitting.min()) * pos_value_weights[pos][0]
    current_players['rank_' + str(pos)] = current_players['wa_projection_' + str(pos)] / current_players['wa_projection_' + str(pos)].max()
    
    current_players['pot_wa_projection_' + str(pos)] = (fielding - fielding.min()) / (fielding.max() - fielding.min()) * pos_value_weights[pos][1] + (hitting_pot - hitting_pot.min()) / (hitting_pot.max() - hitting_pot.min()) * pos_value_weights[pos][0]
    current_players['rank_prospect_' + str(pos)] = current_players['pot_wa_projection_' + str(pos)] / current_players['pot_wa_projection_' + str(pos)].max()

current_players['rank_max'] = current_players[['rank_' + str(i) for i in range(1, 10)]].fillna(0).apply(lambda x: x.max(), axis = 1)
current_players['rank_prospect_max'] = current_players[['rank_prospect_' + str(i) for i in range(1, 10)]].fillna(0).apply(lambda x: x.max(), axis = 1)

In [ ]:
# Preset player queries
fa = current_players.query('free_agent == 1 and draft_eligible == 0')
# trade_block
draft = current_players.query('hsc_status in (4, 5, 6, 9, 10)')
org = current_players.query('organization_id == 164')

relievers = current_players.query('role in (12, 13)')
starters = current_players.query('role == 11')

p = [relievers, starters]

for pos in range(2, 10):
    positional_query = current_players.query('position == ' + str(pos))
    p.append(positional_query)

# Generate Organization Depth Chart (Player Counts)

In [ ]:
levels = {
        200: 'MLB',
        201: 'AAA',
        202: 'AA',
        203: 'A',
        -200: 'Intl Comp',
        204: 'Rookie'
}

pos_dict = {
        (1, 11): 'Starter',
        (1, 12): 'Reliever',
        (1, 13): 'Reliever',
        (2, 0): 'Catcher',
        (3, 0): 'First Base',
        (4, 0): 'Second Base',
        (5, 0): 'Third Base',
        (6, 0): 'Shortstop',
        (7, 0): 'Left Field',
        (8, 0): 'Center Field',
        (9, 0): 'Right Field',
}

def org_chart_counts():
    p = pd.read_csv(path + 'players.csv').query('league_id > -1 and injury_is_injured == 0 and organization_id == ' + str(my_team_id))
    my_org = p.merge(current_players[['player_id', 'name'] + woba_projections + fip_projections + zr_projections])

    my_org['Position'] = my_org[['position', 'role']].apply(lambda info : pos_dict[tuple(info)], axis = 1)
    my_org.sort_values(['position', 'Position'], ascending = [True, True], inplace = True)

    my_org['Level'] = np.vectorize(levels.get)(my_org['league_id'])

    df = my_org.pivot_table(index = ['league_id', 'Level'], columns = 'Position', values = 'player_id', aggfunc = 'count', fill_value = 0, margins = True, margins_name = 'Total').droplevel(0)
    df = df[['Starter', 'Reliever', 'Catcher', 'First Base', 'Second Base', 'Shortstop', 'Third Base', 'Left Field', 'Center Field', 'Right Field', 'Total']]
    return df

# Amateur Draft

In [ ]:
def amateur_draft():
    remaining_draft = pd.read_csv(path[0:-4] + 'mlb_draft_pool.csv', skiprows = [1, 2]).reset_index()
    draft = current_players.merge(remaining_draft[['index']], how = 'inner', left_on = 'player_id', right_on = 'index')
    pos_dict = {1:'Pitcher', 2:'Catcher', 3:'First Base', 4:'Second Base', 5:'Third Base', 6:'Shortstop', 7:'Left Field', 8:'Center Field', 9:'Right Field'}
    
    green_cmap = plt.get_cmap('Blues')
    cmap = plt.get_cmap('Blues')
    red_cmap = plt.get_cmap('Reds')
    outline_colors = ['red', 'black', 'green']

    plt.figure(figsize=(35, 15))
    plt.suptitle('Draft Eligible - Top Batters and Fielders', size = 40)

    for pos in range(2, 10):
        plt.subplot(2, 4, pos - 1).set_facecolor('aliceblue')
        plt.title(pos_dict[pos], size = 25)
        pos = str(pos)

        d = draft.query('position == ' + pos)
        bt = d.sort_values('woba+_projection_pot', ascending = False).head(5)
        fd = d.sort_values('zr_projection_' + pos, ascending = False).head(5)
        d = pd.concat([bt, fd]).drop_duplicates()
        rescaled_ages = (d['age'] - draft['age'].min()) / (draft['age'].max() - draft['age'].min())
        plt.scatter(x = d['woba+_projection_pot'], y = d['zr_projection_' + pos], s = 150, c = cmap(rescaled_ages), edgecolor = [outline_colors[i] for i in d['WE_int']], linewidths = 2)

        plt.xlim(plt.xlim())
        plt.ylim(plt.ylim())
        plt.plot(plt.xlim(), [0, 0], linestyle = 'dashed', color = 'midnightblue', alpha = 0.5)
        plt.plot([100, 100], plt.ylim(), linestyle = 'dashed', color = 'midnightblue', alpha = 0.5)

        for i, player in enumerate(d['name']):
            x_loc = d['woba+_projection_pot'].iloc[i]
            y_loc = d['zr_projection_' + pos].iloc[i]

            plt.annotate(player, (x_loc, y_loc), (x_loc + 0.1, y_loc - 0.02), size = 15)


    plt.figure(figsize=(35, 10))
    plt.suptitle('Draft Eligible - Top Pitchers', size = 40)

    # Starting Pitchers
    plt.subplot(1, 2, 1).set_facecolor('aliceblue')
    plt.title('Starters', size = 30)
    d = draft.query('role == 11').sort_values('fip-_projection_pot', ascending = False).tail(20)
    rescaled_ages = (d['age'] - draft['age'].min()) / (draft['age'].max() - draft['age'].min())
    plt.barh(d['name'], d['fip-_projection_pot'], color = cmap(rescaled_ages), edgecolor = [outline_colors[i] for i in d['WE_int']], linewidth = 3.5)
    plt.yticks(fontsize = 20)

    # Relievers
    plt.subplot(1, 2, 2).set_facecolor('aliceblue')
    plt.title('Relievers', size = 30)
    d = draft.query('role > 11').sort_values('fip-_projection_pot', ascending = False).tail(20)
    rescaled_ages = (d['age'] - draft['age'].min()) / (draft['age'].max() - draft['age'].min())
    plt.barh(d['name'], d['fip-_projection_pot'], color = cmap(rescaled_ages), edgecolor = [outline_colors[i] for i in d['WE_int']], linewidth = 3.5)
    plt.yticks(fontsize = 20)

    plt.tight_layout()
    plt.show()

In [ ]:
# age, level, performance

# x axis - projected potential woba $$$
# y axis - actual woba in current league  $$$
# color - level $$$
# edgecolor - projected fielding
# cmap - age $$$
# size - 
# marker - 
# different graphs - positions

def farm_system_hitters(team_search = 'Austin'):
    search_team_id = teams.query('name == "' + str(team_search) + '" or nickname == "' + str(team_search) + '" or team == "' + str(team_search) + '"').reset_index().at[0, 'team_id']
    search_team_name = teams.query('name == "' + str(team_search) + '" or nickname == "' + str(team_search) + '" or team == "' + str(team_search) + '"').reset_index().at[0, 'team']
    d = current_players.query('organization_id == ' + str(search_team_id))

    age_groups = ['16 - 21', '22 - 25', '26 - 29', '30 - 33', '34+']
    lvls = {-1: 'Reds', 104: 'Oranges', 103: 'YlOrBr', 102: 'Purples', 101: 'Blues', 100: 'Greens', 1: 'Greys'}

    bat_stats = pd.read_csv(path + 'players_career_batting_stats.csv').query('split_id == 1 and year == ' + str(current_year)).merge(current_players[['player_id', 'league_id', 'organization_id']], how = 'right', on = ['player_id', 'league_id']).query('organization_id == ' + str(search_team_id))
    bat_stats['s'] = bat_stats['h'] - bat_stats['hr'] - bat_stats['t'] - bat_stats['d']
    bat_stats['woba'] = (bat_stats['bb'] * 0.69 + bat_stats['hp'] * 0.72 + bat_stats['s'] * 0.89 + bat_stats['d'] * 1.27 + bat_stats['t'] * 1.62 + bat_stats['hr'] * 2.1) / (bat_stats['ab'] + bat_stats['bb'] - bat_stats['ibb'] + bat_stats['sf'] + bat_stats['hp'])
    d = d.merge(bat_stats[['player_id', 'league_id', 'woba']], how = 'inner', on = ['player_id', 'league_id']).fillna(0)

    plt.figure(figsize=(35, 20))
    plt.suptitle(search_team_name  + ' Farm System - Hitters', size = 40, y = 0.95)

    for pos in range(2, 10):
        plt.subplot(2, 4, pos - 1)
        plt.title('Position - ' + str(pos), size = 20)
        df = d.query('position == ' + str(pos))
        plt.scatter(data=df, x='woba+_projection_pot', y='woba', s = 200, c = [plt.get_cmap(lvls[l], 5)(age_groups.index(a)) for l, a in df[['league_id', 'Age Group']].values], edgecolor = 'black', linewidth = 2)

        plt.xlim(plt.xlim())
        plt.ylim(plt.ylim())
        plt.plot([100, 100], plt.ylim(), linestyle = 'dashed', color = 'lightblue')
        plt.plot(plt.xlim(), [0.3, 0.3], linestyle = 'dashed', color = 'lightblue')

        for i, player in enumerate(df['name']):
            xloc = df['woba+_projection_pot'].iloc[i]
            yloc = df['woba'].iloc[i]
            x_buffer = (plt.xlim()[1] - plt.xlim()[0]) * 0.03

            plt.annotate(player, (xloc, yloc), (xloc + x_buffer, yloc), size = 14)
    plt.show()

In [ ]:
def calculate_off_stats(df):
    df['s'] = df['h'] - df['d'] - df['t'] - df['hr']
    df['woba'] = (df['bb'] * 0.69 + df['hp'] * 0.72 + df['s'] * 0.89 + df['d'] * 1.27 + df['t'] * 1.62 + df['hr'] * 2.1) / (df['ab'] + df['bb'] - df['ibb'] + df['sf'] + df['hp'])
    df['avg'] = df['h'] / df['ab']
    df['obp'] = (df['h'] + df['bb'] + df['hp']) / (df['ab'] + df['bb'] + df['hp'] + df['sf'])
    df['tb'] = df['s'] + df['d'] * 2 + df['t'] * 3 + df['hr'] * 4
    df['slg'] = df['tb'] / df['ab']
    df['ops'] = df['obp'] + df['slg']
    df['iso'] = df['slg'] - df['avg']
    df['runs_created'] = (df['h'] + df['bb'] + df['hp'] - df['cs'] - df['gdp']) * (df['tb'] + 0.26 * (df['bb'] + df['hp'] - df['ibb']) + 0.52 * (df['sh'] + df['sf'] + df['sb'])) / (df['ab'] + df['bb'] + df['hp'] + df['sh'] + df['sf'])
    
def calculate_pit_stats(df):
    df['k_rate'] = df['k'] / df['bf']
    df['bb_rate'] = df['bb'] / df['bf']
    df['hr_rate'] = df['hra'] / df['bf']
    df['fip'] = (df['hra'] * 13 + (df['bb'] + df['hp']) * 3 - df['k'] * 2) / df['ip'] + 3.2
    df['era'] = df['er'] / df['ip'] * 9
    df['babip'] = (df['ha'] - df['hra']) / (df['ab'] - df['k'] - df['hra'] + df['sf'])

In [ ]:
def minor_league_performance():
    df = pd.read_csv(path + 'players_career_batting_stats.csv').drop('position', axis = 'columns').merge(players[['player_id', 'league_id', 'name', 'organization_id', 'position']], how = 'inner', on = ['player_id', 'league_id']).query('organization_id == 3 and league_id != 1 and year == 2026 and split_id == 1')
    calculate_off_stats(df)
    df.sort_values(['level_id', 'woba'], ascending = [False, True], inplace = True)

    lvls = {6: 'indianred', 5: 'orange', 4: 'palevioletred', 3: 'blueviolet', 2: 'royalblue'}
    lvl_names = {6: 'Rookie', 5: 'A-', 4: 'A', 3: 'AA', 2: 'AAA'}

    plt.figure(figsize=(35, 20))
    plt.suptitle('Minor League Performance', size = 40, y = 0.92)
    plt.subplot(1, 2, 1)
    plt.title('Hitters', size = 25)

    for l in lvls:
        d = df.query('level_id == ' + str(l))
        plt.barh(d['name'], d['woba'], color = lvls[l], label = lvl_names[l])

    plt.ylim(plt.ylim())
    plt.plot([0.2, 0.2], plt.ylim(), linestyle = 'dashed', color = 'red', label = '0.200')
    plt.plot([0.3, 0.3], plt.ylim(), linestyle = 'dashed', color = 'orange', label = '0.300')
    plt.plot([0.4, 0.4], plt.ylim(), linestyle = 'dashed', color = 'forestgreen', label = '0.400')

    plt.legend(loc = 'lower right', fontsize = 20)

    df = pd.read_csv(path + 'players_career_pitching_stats.csv').merge(players[['player_id', 'league_id', 'name', 'organization_id']], how = 'inner', on = ['player_id', 'league_id']).query('organization_id == 3 and league_id != 1 and year == 2026 and split_id == 1')
    calculate_pit_stats(df)
    df.sort_values('fip', ascending = False, inplace = True)

    plt.subplot(1, 2, 2)
    plt.title('Pitchers', size = 25)

    for l in lvls:
        d = df.query('level_id == ' + str(l))
        plt.barh(d['name'], d['fip'], color = lvls[l], label = lvl_names[l])

    plt.ylim(plt.ylim())
    plt.plot([2, 2], plt.ylim(), linestyle = 'dashed', color = 'forestgreen', label = '2.00')
    plt.plot([3.5, 3.5], plt.ylim(), linestyle = 'dashed', color = 'orange', label = '3.50')
    plt.plot([5, 5], plt.ylim(), linestyle = 'dashed', color = 'red', label = '5.00')

    plt.legend(loc = 'lower right', fontsize = 20)
    plt.show()

In [ ]:
def org_vs_fa_tb(pos, top = 20):
    pos = str(pos)
    columns = ['name', 'age', 'position', 'team', 'org', 'level', 'current_salary', 'draft_eligible', 'free_agent', 'woba_projection', 'zr_projection_' + pos, 'rank_' + pos]

    org = current_players.query('org == "Denver Silverbacks"')[columns]
    fa = current_players.query('free_agent == 1 and draft_eligible == 0 and position == ' + pos)[columns]
    trade_block = current_players.query('(trade_status == 2 or is_on_waivers == 1) and position == ' + pos)[columns]

    return pd.concat([org, fa, trade_block]).sort_values('rank_' + pos, ascending = False).head(top)